# LZ78 Sequential Probability Assignment: Python Implementation
This code is associated with the paper [A Family of LZ78-based Universal Sequential Probability Assignments](https://arxiv.org/abs/2410.06589).

This codebase is in Python, which is more popular than Rust. This Python codebase gives users the option to experiment more comfortably with implementation modifications at the cost of slower runtime (on the order of about 3-5x).

## Setup

1. (Optional) Set up and activate a virtual environment for this project.
2. Install the `lz78_python` package: `pip install --editable .`. Note that the `--editable` option allows you to implementation modifications to propagate to the package without having to rerun `pip install .`.

You should be all set! This tutorial will walk you through the functionalities that the Python codebase offers parallel to the Rust codebase functionalities.

## Imports

In [ ]:
!pip install lorem requests

In [ ]:
from lz78_python.utils.CharacterMap import CharacterMap
from lz78_python.naive.encoder import LZ78_encode
from lz78_python.naive.decoder import LZ78_decode
from lz78_python.streamed.encoder import BlockLZ78Encoder
from lz78_python.spa.encoder import LZ78SPA
import lorem, bitarray, requests
from os import makedirs
import numpy as np

## 1. Sequences

This class does not explicitly exist in this version of the codebase. We can directly use Python lists for integer sequences and Python strings for character sequences. 

### 1.1 Example: Integer Sequence

We will not go into depth with this example, given that you should be able to recreate the same behaviors through Python list.

However, Python list does not have a direct method to check the number of unique symbols (ie. alphabet size), but you can always do `len(set(lst))` for that functionality.

### 1.2 `CharacterMap`

Underlying logic assumes an integer representation of a sequence, so we need a way to map strings to integer-based sequences ranging from `0` to `A-1`.

The `CharacterMap` class maps characters in a string to integer values in a contiguous range, so that a string can be used as an individual sequence.
It has the capability to **encode** a string into the corresponding integer representation, and **decode** a list of integers into a string.

Inputs:
- data: a string consisting of all of the characters that will appear in the character map. For instance, a common use case is:
    ```
    charmap = CharacterMap("abcdefghijklmnopqrstuvwxyz")
     ```

In [ ]:
# generate some dummy data and make a character map
s = " ".join(([lorem.paragraph() for _ in range(10)]))
charmap = CharacterMap(s)

#### Instance method: `encode`
Takes a string and returns the corresponding integer representation.

In [ ]:
charmap.encode("lorem ipsum")

It errors if any characters to be encoded are not in the alphabet.

In [ ]:
charmap.encode("hello world")

#### Instance method: `filter_string`
Takes a string and removes any characters that are not present in the character mapping.
This is useful if you have some text with special characters, and you don't want the special characters to be in the alphabet.

In [ ]:
charmap.filter_string("hello world. Lorem ipsum! @#$%^&*()")

#### Instance method: `decode`
Decodes an integer representation of a string into the string itself

In [ ]:
charmap.decode(charmap.encode("lorem ipsum"))

#### Instance method: `alphabet_size`
Returns how many characters can be represented by the character mapping

In [ ]:
charmap.alphabet_size()

### 1.3 Example: Character Sequence

A string-based sequence in Python is represented simply as a string. We will need to define a `CharacterMap` for this sequence and pass it into all sequence-related functions.

In [ ]:
charmap = CharacterMap("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ. ?,")
charseq = " ".join(([lorem.paragraph() for _ in range(1000)]))

To get the integer representation of the corresponding characters, we use the `CharacterMap.encode` function, but indexing a character sequence directly in this implementation will return the characters of the character sequence.

In [ ]:
charmap.encode(charseq[100:130])

In [ ]:
charseq[100:130], charmap.decode(charmap.encode(charseq[100:130]))

Since we do not have a `Sequence` object, there is no `CharacterMap` explicitly tied in with the Python string (that serves as the character sequence in this implementation). The burden will be on the user to keep track of the `CharacterMap`'s corresponding to the strings.

## 2. LZ78 Compression

The `LZ78Encoder` object performs plain LZ78 encoding and decoding, as described in "Compression of individual sequences via variable-rate coding" (Ziv, Lempel 1978).

### 2.1 `CompressedSequence` tuple object

The `CompressedSequence` object stores an encoded bitstream and compression ratio of the encoding compared the original uncompressed data. `CompressedSequence` objects cannot be instantiated directly, but rather are returned by `LZ78Encoder.encode`.

The main functionality is:
1. Getting the compression ratio as `(encoded size) / (uncompressed len * log A)`,
    where A is the size of the alphabet.
2. Getting a `bitarray.bitarray` representing this object, so that the compressed
    sequence can be stored to a file

In [ ]:
data = " ".join(([lorem.paragraph() for _ in range(10_000)]))
charmap = CharacterMap(data)

#### `LZ78Encoder` Instance method: `encode`
Performs LZ78 encoding on an individual sequence, and returns a `LZ78Output` object.

In [ ]:
encoded = LZ78_encode(data, custom_char_map=charmap)

#### `Compressed Sequence` Tuple attribute: `compression_ratio`

In [ ]:
encoded.compression_ratio

#### Saving a `CompressedSequence` tuple object

This is a parallel to the `to_bytes` and `from_bytes` functionality offered by the Rust implementation. We use the underlying `bitarray.bitarray` representation to directly write the LZ78 encoded bits to a file and read from it. 

However, unlike the Rust `CompressedSequence` implementation, reading the data from the file does not return the original `CompressedSequence` tuple object; it only returns the encoded `bitarray.bitarray`. (So, we would be losing out on immediate compression ratio information.)

In [ ]:
# NOTE: reading bitarray from file may include additional 0's 
# because of padding when writing the data to file, so we 
# should also track length and apply it when loading the bits
encoded_bitlength = len(encoded.bits)
makedirs("test_data", exist_ok=True)
with open("test_data/saved_encoded_sequence.bin", 'wb') as file:
    encoded.bits.tofile(file)

Now, let's read the compressed sequence from the file and decode it.

In [ ]:
bits = bitarray.bitarray()
with open("test_data/saved_encoded_sequence.bin", 'rb') as file:
    bits.fromfile(file)
bits = bits[:encoded_bitlength]

In [ ]:
decoded = LZ78_decode(
    bits,
    alphabet_size=charmap.alphabet_size(), 
    return_str=True,
    custom_char_map=charmap,
)

In [ ]:
assert decoded == data

### 2.3 Block-Wise Compression
Sometimes, it might be useful to loop through blocks of data and perform LZ78 encoding on each block (e.g., if you need to do data processing before LZ78 compression and want to have some sort of pipeline parallelism).

The `BlockLZ78Encoder` has this functionality: you can pass in the input sequence to be compressed in chunks, and the output (`encoder.get_encoded_sequence()`) is as if the full concatenated sequence was passed in to an LZ78 encoder.

In [ ]:
charmap = CharacterMap("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ. ,?")

In [ ]:
encoder = BlockLZ78Encoder(
    alphabet_size=charmap.alphabet_size(),
    input_is_string=True,
    custom_char_map=charmap
)

#### Instance method: `encode_block`
Encodes a block using LZ78, starting at the end of the previous block.

All blocks must be over the same alphabet, or else the call to `encode_block` will error.

In [ ]:
for _ in range(1000):
    encoder.encode_block(lorem.paragraph())

In [ ]:
encoder.encode_block([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

#### Instance method: `get_encoded_sequence`
Returns the compressed sequence, which is equivalent to the output of `LZ78Encoder.encode` on the concatenation of all inputs to `encode_block` thus far.

In [ ]:
encoder.get_encoded_sequence()

Instance method: `compression_ratio`

The encoder contains information about the `compression_ratio`, not the encoded bit `bitarray.bitarray`, unlike the `CompressedSequence` object of the Rust implementation.

In [ ]:
encoder.compression_ratio()

#### Method: `LZ78_decode`
Decompresses the compressed sequence that has been constructed thus far. This is the same method used for decoding the "naive" (all-in-one-go) encoding. This is not an instance method, unlike the `decode` function of the Rust implementation.

In [ ]:
decoded = LZ78_decode(
    encoder.get_encoded_sequence(),
    alphabet_size=charmap.alphabet_size(), 
    return_str=True,
    custom_char_map=charmap,
)
print(decoded[376:400])
charmap.encode(decoded[376:400])

## 3. LZ78 Sequential Probability Assignment (SPA)
The `LZ78SPA` class is the implementation of the family of sequential probability assignments discussed in [A Family of LZ78-based Universal Sequential Probability Assignments](https://arxiv.org/abs/2410.06589), for Dirichlet priors.
In this section, `gamma` refers to the Dirichlet parameter.

Under this prior, the sequential probability assignment is an additive
perturbation of the emprical distribution, conditioned on the LZ78 prefix
of each symbol (i.e., the probability model is proportional to the
number of times each node of the LZ78 tree has been visited, plus gamma).

This SPA has the following capabilities:
- training on one or more sequences,
- log loss ("perplexity") computation for test sequences,
- SPA computation (using the LZ78 context reached at the end of parsing
    the last training block),
- sequence generation.

Note that the LZ78SPA does not perform compression; you would have to use
a separate BlockLZ78Encoder object to perform block-wise compression.

### 3.1 Example: LZ78 SPA on Markov Data

We will use the Markov probability source used in [(Rajaraman et al, 2024)](https://arxiv.org/pdf/2404.08335), where the transition probability depends solely on $x_{t-k}$.
Specifically, $x_t = x_{t-k}$ with probability $0.9$, and otherwise $x_t$ is picked uniformly at random from the rest of the alphabet.

The SPA works best when the alphabet size is $2$, but you can try out other alphabet sizes too.

First, we define some helper functions for generating the data (don't worry about understanding these; they are irrelevant to understanding the SPA itself).

In [ ]:
# Helper methods for generating data; feel free run the cell without
# reading the code
def sample_index_from_dist(probabilities):
    cdf = np.cumsum(probabilities)
    cdf[-1] = 1 # in case of FP error
    return int(np.where(np.random.random() < cdf)[0][0])

def entropy(probs):
    return sum([-x * np.log2(x) for x in probs if x > 0])

def get_stationary_dist(transition_probabilities):
    eigvals, eigvecs = np.linalg.eig(transition_probabilities.T)
    # all eigenvalues will be <= 1, and one will be =1
    stationary_dist = eigvecs[:, np.argmax(eigvals)]
    return stationary_dist / sum(stationary_dist)

def entropy_rate(transition_probabilities):
    stationary_dist = get_stationary_dist(transition_probabilities)
    return sum([prob * entropy(transition_probabilities[i]) 
                for i, prob in enumerate(stationary_dist)])

Generate some data to pass through the SPA:

In [ ]:
## You can change these
ALPHABET_SIZE = 2
PEAK_PROB = 0.9
K = 5
N = 1_000_000
N_TEST = 10_000

In [ ]:
# Build data array; feel free to ignore this code and just run the cell
transition_probabilities = np.eye(ALPHABET_SIZE) * PEAK_PROB + \
    (np.ones((ALPHABET_SIZE, ALPHABET_SIZE)) - np.eye(ALPHABET_SIZE)) * (1 - PEAK_PROB) / (ALPHABET_SIZE - 1)
start_prob = np.ones(ALPHABET_SIZE) / ALPHABET_SIZE

data = np.zeros(N, dtype=int)
for i in range(K):
    data[i] = sample_index_from_dist(start_prob)
for i in range(K,N):
    data[i] = sample_index_from_dist(transition_probabilities[data[i-K]])

### Class: `LZ78SPA`

Observe the possible inputs to the LZ78SPA class. It is slightly different than that of the Rust implementation. For instance, you must provide a `CharacterMap` to the SPA in this case (if dealing with character sequences).

In [ ]:
help(LZ78SPA)

#### Instance method: `train_on_block`

Use a block of data to update the SPA. If `include_prev_context` is
true, then this block is considered to be from the same sequence as
the previous. Otherwise, it is assumed to be a separate sequence, and
we return to the root of the LZ78 prefix tree at the start of training.

It returns the self-entropy log loss incurred while processing this
sequence.

In [ ]:
spa = LZ78SPA()
spa.train_on_block(data[:-N_TEST])

#### Instance method: `compute_test_loss`
After training a SPA, you can compute the log loss of a test sequence.

In [ ]:
spa.compute_test_loss(
    data[-N_TEST:], 
    len(data[-N_TEST:]),
    include_prev_context=True
) / N_TEST

#### Instance method: `get_normalized_log_loss`
Gets the normalized self-entropy log loss incurred from training the SPA thus far.

In [ ]:
spa.get_normalized_log_loss()

#### Instance method: `get_prob_for_next_symbol`

Computes the SPA for the specified symbol at the alphabet, using the LZ78 context reached at the end of parsing the last training block. To achieve the functionality of the `compute_spa_at_current_state` Rust method, we can apply a list comprehension over calculating `get_prob_for_next_symbol` over the symbols in the alphabet

In this case, the list comprehension will return a two-element list, where the first element is the estimated probability that the next symbol is $0$ and the second is the estimated probability that the next symbol is $1$.

In [ ]:
[spa.get_prob_for_next_symbol(i) for i in range(ALPHABET_SIZE)]

Unlike the Rust implementation, the Python implementation does not offer a `to_bytes` instance method for the `LZ78SPA` class and by extension a `spa_from_bytes` method to recover the SPA.

### 3.2 Example: Text Generation

Let's use the LZ78 SPA to generate some text based on Sherlock Holmes novels.

This requires the `requests` library and an internet connection.
If you don't have either, you can perform the same experiment any text you'd like, including the lorem ipsum text from the beginning of this tutorial.
Just make sure you have enough training data (e.g., the Sherlock novel used for this example is 500 kB).

In [ ]:
text = requests.get("https://www.gutenberg.org/cache/epub/1661/pg1661.txt").text

Let's define our own character map and filter the text based on this.

In [ ]:
charmap = CharacterMap("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ. ,?\n\"';:\t-_")
filtered_text = charmap.filter_string(text)

Next, train the SPA.

In [ ]:
spa = LZ78SPA(
    alphabet_size=charmap.alphabet_size(), 
    gamma=0.2, 
    custom_char_map=charmap,
    input_is_string=True,
)
spa.train_on_block(filtered_text)

#### Instance method: `generate_data`
Generates a sequence of data, using temperature and top-k sampling (see
the "Experiments" section of [Sagan and Weissman 2024] for more details).

Inputs:
- **len**: number of symbols to generate
- **min_context** (optional): the SPA tries to maintain a context of at least a
    certain length at all times. So, when we reach a leaf of the LZ78
    prefix tree, we try traversing the tree with different suffixes of
    the generated sequence until we get a sufficiently long context
    for the next symbol.
- **temperature** (optional): a measure of how "random" the generated sequence is. A
    temperature of 0 deterministically generates the most likely
    symbols, and a temperature of 1 samples directly from the SPA.
    Temperature values around 0.1 or 0.2 function well.
- **top_k** (optional): forces the generated symbols to be of the top_k most likely
    symbols at each timestep.
- **seed_data** (optional): you can specify that the sequence of generated data
be the continuation of the specified sequence.

Returns a tuple of the generated sequence and that sequence's log loss,
or perplexity.

Errors if the SPA has not been trained so far, or if the seed data is
not over the same alphabet as the training data.

In [ ]:
(generated, loss) = spa.generate_data(
    500,
    min_context=5,
    temperature=0.1,
    top_k=5,
    seed_data="This "
)
generated = "This " + generated

Right now, the generated text will not appear very high-quality for a few reasons:
1. We don't have a lot of training data. LZ78-based algorithms generally take a lot of data to train; empirically, the SPA does better with many megabytes of data, at least.
2. The `min_context` parameter is being used very heuristically, and could potentially be forcing us to the bottom of the LZ78 tree, where we don't have a lot of data, making the SPA closer to uniform. If you see long gibberish strings, try decreasing `min_context`.

In [ ]:
for i in range(0, len(generated), 80):
    print(generated[i:i+80])